In [2]:
%tensorflow_version 1.x
from tensorflow.contrib.layers import batch_norm, flatten
from keras.models import model_from_json
import os
import openpyxl
import sys
import tensorflow as tf
import numpy as np
import math
from keras.callbacks import EarlyStopping
import random
from random import seed
from keras import backend as K
import gc
seed(1)

#os.environ["CUDA_VISIBLE_DEVICES"] = str(2)  # gpu_id

from google.colab import drive
drive.mount('/content/gdrive')

from tensorflow.python.client import device_lib

device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))
file_path = '/content/gdrive/My Drive/Colab Notebooks/k-fold/'

def load_model(model_name):
  json_file = open(str(model_name),'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model = tf.keras.models.model_from_json(loaded_model_json)
  return model

def hardVoting(models):
  name_1 = models[0]
  name_2 = models[1]
  name_3 = models[2]
  name_4 = models[3]
  name_5 = models[4]


  model_cnt = [1,2,3,4,5,6,7,8,9,10]
  test_accurancy_1 = np.zeros((10,))
  test_accurancy_2 = np.zeros((10,))
  test_accurancy_3 = np.zeros((10,))
  test_accurancy_4 = np.zeros((10,))
  test_accurancy_5 = np.zeros((10,))

  ensemble_accurancy = np.zeros((10,))

  loaded_model_1 = load_model(file_path + name_1 + '1.json')
  loaded_model_2 = load_model(file_path + name_2 + '1.json')
  loaded_model_3 = load_model(file_path + name_3 + '1.json')
  loaded_model_4 = load_model(file_path + name_4 + '1.json')
  loaded_model_5 = load_model(file_path + name_5 + '1.json')


  loaded_model_1.build(input_shape=(256,22))
  loaded_model_2.build(input_shape=(256,22))
  loaded_model_3.build(input_shape=(256,22))
  loaded_model_4.build(input_shape=(256,22))
  loaded_model_5.build(input_shape=(256,22))

  
  for i in model_cnt:
    print(str(i)+'-fold test start')

    model_weight_1 = file_path + name_1 + str(i) + '_' + str(i) + '.h5'
    model_weight_2 = file_path + name_2 + str(i) + '_' + str(i) + '.h5'
    model_weight_3 = file_path + name_3 + str(i) + '_' + str(i) + '.h5'
    model_weight_4 = file_path + name_4 + str(i) + '_' + str(i) + '.h5'
    model_weight_5 = file_path + name_5 + str(i) + '_' + str(i) + '.h5'


    loaded_model_1.load_weights(model_weight_1)
    loaded_model_2.load_weights(model_weight_2)
    loaded_model_3.load_weights(model_weight_3)
    loaded_model_4.load_weights(model_weight_4)
    loaded_model_5.load_weights(model_weight_5)


    loaded_model_1.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    loaded_model_2.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])  
    loaded_model_3.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']) 
    loaded_model_4.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])  
    loaded_model_5.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']) 

    print("Loaded model from disk")

    x_train = np.loadtxt(file_path+'x_train'+str(i)+'(1).txt')
    y_train = np.loadtxt(file_path+'y_train'+str(i)+'(1).txt')
    x_test = np.loadtxt(file_path+'x_test'+str(i)+'(1).txt')
    y_test = np.loadtxt(file_path+'y_test'+str(i)+'(1).txt')

    test_acc_1 = loaded_model_1.evaluate(x_test, y_test)
    test_acc_2 = loaded_model_2.evaluate(x_test, y_test)
    test_acc_3 = loaded_model_3.evaluate(x_test, y_test)
    test_acc_4 = loaded_model_4.evaluate(x_test, y_test)
    test_acc_5 = loaded_model_5.evaluate(x_test, y_test)


    test_accurancy_1[i-1] = test_acc_1[1]
    test_accurancy_2[i-1] = test_acc_2[1]
    test_accurancy_3[i-1] = test_acc_3[1]
    test_accurancy_4[i-1] = test_acc_4[1]
    test_accurancy_5[i-1] = test_acc_5[1]


    #hard Voting
    tmp_1 = np.argmax (np.asarray(loaded_model_1.predict(x_test)) , axis =1)
    tmp_2 = np.argmax (np.asarray(loaded_model_2.predict(x_test)) , axis =1)
    tmp_3 = np.argmax (np.asarray(loaded_model_3.predict(x_test)) , axis =1)
    tmp_4 = np.argmax (np.asarray(loaded_model_4.predict(x_test)) , axis =1)
    tmp_5 = np.argmax (np.asarray(loaded_model_5.predict(x_test)) , axis =1)


    tmp = np.around((tmp_1 + tmp_2 + tmp_3 + tmp_4 + tmp_5 )/5)
    print(tmp)
    print(y_test)
    
    ensemble_acc = np.sum(tmp == y_test)/80
    ensemble_accurancy[i-1] = ensemble_acc
    print('ensemble_acc : ' + str(ensemble_acc))

  print(ensemble_accurancy)
  ensemble_average=np.sum(ensemble_accurancy)/10
  print('ensemble average : ' + str(ensemble_average))
  del loaded_model_1
  del loaded_model_2
  del loaded_model_3
  del loaded_model_4
  del loaded_model_5

  gc.collect()
  K.clear_session()
  return ensemble_average

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Found GPU at: 


In [3]:
name = ['64_128_64_0.6_','32_64_64_0.6_','64_32_16_8_4_0.6_','32_32_16_8_4_0.6_','64_64_32_16_8_0.6_','64_64_16_8_4_0.6_','64_32_32_8_4_0.6_','64_32_32_16_4_0.6_']
max_avg=0
max_models=[]

ensemble_file = open(file_path + 'ensemble_file(5).txt',"w")
cnt=0

def dfs(num,models,start,finish):
  global max_avg
  global max_models
  global cnt
  if start > finish:
    return
  if num == 5:
    print(models)
    avg=hardVoting(models)
    ensemble_file.write(str(models)+'  '+ str(avg) + '\n')
    if max_avg < avg:
      cnt=1
      max_avg = avg
      max_models=models[:]
    elif max_avg == avg:
      cnt+=1
    print(max_avg)
    return

  else:
    for i in range(start,finish):
      models.append(name[i])
      dfs(num+1,models,i+1,finish)
      models.pop()

dfs(0,[],0,len(name))

print('max average : ' , max_avg)
print('max modles : ' , max_models)
print('cnt : ',cnt)
ensemble_file.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0.
 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 1. 1. 1. 1. 1.]
[0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 0.]
ensemble_acc : 0.7625
7-fold test start
Loaded model from disk
80/80 [==============================] - 0s 3ms/sample - loss: 1.1901 - acc: 0.7875
[1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1.
 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 0. 1. 1.]
[1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1.
 1. 1. 1.

In [4]:
name = ['64_32_16_8_4_0.6_','32_32_16_8_4_0.6_','64_64_32_16_8_0.6_','64_64_16_8_4_0.6_','64_32_32_8_4_0.6_','64_32_32_16_4_0.6_']
hardVoting(name)


1-fold test start
Loaded model from disk
80/80 [==============================] - 0s 3ms/sample - loss: 2.7116 - acc: 0.7875
[1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0.
 1. 0. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 1. 0.]
ensemble_acc : 0.8
2-fold test start
Loaded model from disk
80/80 [==============================] - 0s 2ms/sample - loss: 1.9390 - acc: 0.7750
[0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0.
 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 0. 

0.77625

In [0]:
ensemble_file.close()